## Exercises:
* ##### E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
* ##### E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
* ##### E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
* ##### E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
* ##### E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
* ##### E06: meta-exercise! Think of a fun/interesting exercise and complete it.

### E01

In [ ]:
# words = ['..'+line.split(',')[1].lower()+'.' for line in open("data/NationalNames.csv", 'r').read().splitlines()][1:]
words = [line.split(',')[1].lower() for line in open("data/NationalNames.csv", 'r').read().splitlines()][1:]

{',': 0,
 '.': 1,
 '5': 2,
 '8': 3,
 '9': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'k': 15,
 'l': 16,
 'm': 17,
 'n': 18,
 'o': 19,
 'p': 20,
 'q': 21,
 'r': 22,
 's': 23,
 't': 24,
 'u': 25,
 'v': 26,
 'w': 27,
 'x': 28,
 'y': 29,
 'z': 30,
 'ب': 31,
 'ث': 32,
 'س': 33,
 'ف': 34,
 'ل': 35,
 'ي': 36}

In [ ]:
for i in range(5):
    word = words[i]
    print(word)
    trigrams = [(word[i:i + 2], word[i + 2]) for i in range(len(word) - 2)]
    print(f"Trigrams: {str(trigrams)}\n-----------")

In [ ]:
words = set(words)

In [ ]:
from collections import Counter

trigrams = []
for word in words:
    trigrams += [(word[i:i + 2], word[i + 2]) for i in range(len(word) - 2)]

trigram_counts = Counter(trigrams, )

In [ ]:
trigrams[:10]

In [ ]:
trigram_counts.most_common()[:20]

In [ ]:
print(len(trigrams))
print(len(trigram_counts))

In [ ]:
alphabet = list('.abcdefghijklmnopqrstuvwxyz')
output_stoi = {s: i for i, s in enumerate(alphabet)}
output_itos = {i: s for i, s in enumerate(alphabet)}
# print(output_stoi)
# print(output_itos)

In [ ]:
input_stoi = {s: i for i, s in enumerate([j + k for j in alphabet for k in alphabet])}
input_itos = {i: s for i, s in enumerate([j + k for j in alphabet for k in alphabet])}
# print(input_stoi)
# print(input_itos)

#### Sampling from a 2d matrix

In [ ]:
import torch

N = torch.zeros(729, 27)
for count in trigram_counts.keys():
    N[input_stoi[count[0]], output_stoi[count[1]]] = trigram_counts[count]

In [ ]:
P = (N + 1) / (N + 1).sum(1, keepdim=True)
P

In [ ]:
g = torch.Generator().manual_seed(0)
for i in range(10):

    out = []
    ix = '..'
    while True:
        p = P[input_stoi[ix]]
        ox = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        # ox = torch.multinomial(torch.ones(27)/27, num_samples=1, replacement=True, generator=g).item()
        output_char = output_itos[ox]
        out.append(output_char)
        ix = ix[1] + output_char
        if ox == 0:
            break
        
    print(''.join(out))


In [ ]:
L = P.log()
nnl = 0
n=0
for w in ['..ahmed.','..rushdi.']:
    for ch1, ch2, ch3 in zip(w[:], w[1:], w[2:]):
        print(f"{ch1+ch2+ch3}: {P[input_stoi[ch1+ch2]][output_stoi[ch3]]:.5f} {(temp := L[input_stoi[ch1+ch2]][output_stoi[ch3]]):.5f} {trigram_counts[(ch1+ch2,ch3)]}")
        nnl -= temp
        n+=1
nnl/n

In [ ]:
X_data, y_data = map(list, zip(*trigrams))
X_data[:10], y_data[:10]

In [ ]:
X = torch.tensor([input_stoi[x] for x in X_data])
y = torch.tensor([output_stoi[y] for y in y_data])

In [ ]:
nll = -sum(L[x,y] for x,y in zip(X, y))/len(X)

In [ ]:
nll

Ahmed Rushdi is an unlikely name w.r.t. the distribution of this dataset as it has higher Negative Log Likelihood than the dataset average.

#### NN trigram implementation

In [ ]:
import torch.nn.functional as F
X_enc = F.one_hot(X).float()
W = torch.randn((729,27), requires_grad = True)
y_enc = F.one_hot(y).float()

In [ ]:
logits = X_enc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
probs.shape

In [ ]:
nll = -(probs[X, y]).log().mean()
nll

In [ ]:
loss = None
for i in range(1,51):
    logits = X_enc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -(probs[X, y]).log().mean()
    W.grad = None
    loss.backward()
    W.data += -90 * W.grad
    print(f"Epoch {i}: loss\t{loss}")